In [24]:
#For installing surprise 
conda install -c conda-forge scikit-surprise

In [17]:
# importing libaries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from surprise import Dataset
from surprise import Reader
from surprise import SVD
import random as rnd

In [8]:
# reading datasets
ratings_data = pd.read_csv('rating.csv')
anime_data = pd.read_csv('anime.csv')
anime_data.head(10)

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
5,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",TV,10,9.15,93351
6,11061,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855
7,820,Ginga Eiyuu Densetsu,"Drama, Military, Sci-Fi, Space",OVA,110,9.11,80679
8,15335,Gintama Movie: Kanketsu-hen - Yorozuya yo Eien...,"Action, Comedy, Historical, Parody, Samurai, S...",Movie,1,9.10,72534
9,15417,Gintama&#039;: Enchousen,"Action, Comedy, Historical, Parody, Samurai, S...",TV,13,9.11,81109


1. all ratings with rating value -1 are removed from the dataset in the preprocessing phase so that the algoritm won't be build based on the -1 values which would be wrong.

In [9]:
# removing -1 ratings
ratings_data = ratings_data[ratings_data.rating != -1]
ratings_data.head(10)

,user_id,anime_id,rating
47,1,8074,10
81,1,11617,10
83,1,11757,10
101,1,15451,10
153,2,11771,10
156,3,20,8
157,3,154,6
158,3,170,9
159,3,199,10
160,3,225,9


In [10]:
# creating data for recommendation alogorithm
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(ratings_data[['user_id', 'anime_id', 'rating']], reader)

2. SVD recommendation algorithm doesn't seem to be slow at least with current hardware.

In [13]:
# training algorithm
svd = SVD(verbose=True, n_epochs=12)
trainset = data.build_full_trainset()
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11


3. below the recomendation function combines data from both datasets to return recomendation info including title.

In [23]:
# recomend random anime to user whose id is user_id. Anime's rating is bigger or equal to min_rating
def recomendAnime(user_id, min_rating):
    animes = list(anime_data['anime_id'].values)
    rnd.shuffle(animes)

    for anime in animes:
        rating = svd.predict(uid=user_id, iid=anime)
        if(rating.est >= min_rating):
            anime_info = anime_data[anime_data['anime_id'] == anime][['anime_id','name', 'genre', 'type', 'episodes', 'rating', 'members']]
            #print info to user
            print(anime_info)
            # return recomendation to user
            return anime_info
        
#test recomendation
recomendAnime(1, 8)
print("----")
recomendAnime(3, 6)
print("----")

      anime_id                            name   genre type episodes  rating  \
9127     30444  Kana Kana Kazoku: Stop Rubella  Comedy  ONA        1    4.14   

      members  
9127       71  
----
      anime_id                         name                   genre type  \
2947       358  Iriya no Sora, UFO no Natsu  Drama, Romance, Sci-Fi  OVA   

     episodes  rating  members  
2947        6    7.12    18254  
----
